In [ ]:
pip install --upgrade imblearn

In [ ]:
pip install glove_python

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
from sklearn import datasets
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge 
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from numpy import mean
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score
import sklearn.metrics as metrics
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegressionCV
import re
import nltk
from nltk.corpus import stopwords
import datetime
import json
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

In [ ]:
df= pd.read_csv('arabica_data_cleaned.csv')

In [ ]:
df

In [ ]:
df1= pd.read_csv('merged_data_cleaned.csv')

In [ ]:
df1

In [ ]:
frames = [df, df1]

result = pd.concat(frames)

In [ ]:
result

In [ ]:
pd.set_option('display.max_columns', None)
result.head()

In [ ]:
result.corr

In [ ]:
result.info()

In [ ]:
result.describe()

In [ ]:
result.isnull().sum()

In [ ]:
result = result.drop(['Lot.Number','Unnamed: 0','In.Country.Partner'],axis=1)

In [ ]:
result['Company'].isna().sum()


In [ ]:
result['Company'].mode()


In [ ]:
result['Country.of.Origin'].isna().sum()

In [ ]:
result = result[result['Country.of.Origin'].notna()]
result

In [ ]:
result = result.drop(['Farm.Name','Mill','ICO.Number','Company','Altitude','Producer'],axis=1)

In [ ]:
result

In [ ]:
result= result.drop(['Number.of.Bags','Bag.Weight'],axis=1)

In [ ]:
result

In [ ]:
result.loc[result["Harvest.Year"] == "2017 / 2018", "Harvest.Year"] = "2018"
result.loc[result["Harvest.Year"] == "2016 / 2017", "Harvest.Year"] = "2017"
result.loc[result["Harvest.Year"] == "2015/2016", "Harvest.Year"] = "2016"
result.loc[result["Harvest.Year"] == "2014/2015", "Harvest.Year"] = "2015"
result.loc[result["Harvest.Year"] == "2013/2014", "Harvest.Year"] = "2014"
result.loc[result["Harvest.Year"] == "2011/2012", "Harvest.Year"] = "2012"

In [ ]:
result

In [ ]:
result = result[result['Harvest.Year'].notna()]
result

In [ ]:
result = result.drop(['Variety'],axis=1)


In [ ]:
result

# EDA

In [ ]:
# Importing pandas and pandas profiling
import pandas as pd
from pandas_profiling import ProfileReport
# Importing the dataset
result = pd.concat(frames)

In [ ]:
profile = ProfileReport(df, title='Titanic Report')
profile

If the altitude increases, acidity will decrease:

In [ ]:
result.altitude_mean_meters.plot(kind = "line", color = "Blue", label = "altitude_mean_meters", linewidth = 1, alpha = 0.5, grid = True , linestyle = ":")
result.Acidity.plot(color = "red", label = "Acidity", linewidth = 1, alpha = 0.5, grid = True , linestyle = "-.")
plt.legend(loc = "upper right")
plt.xlabel("x axis")
plt.ylabel("y axis")
plt.title("Line plot")
plt.show()

Top coffee producing countries

In [ ]:
cdf = result.groupby("Country.of.Origin").size().reset_index(name="Harvest.Year") \
    .sort_values("Harvest.Year", ascending=False).head(20)

fig, ax = plt.subplots(figsize=(8,8))
ch_ = sns.barplot(x="Harvest.Year", y="Country.of.Origin", data=cdf,palette=('mako'), ax=ax)
ch_ = ax.set(xlabel="Production", ylabel="Country of Origin")

In [ ]:
result.groupby(['Country.of.Origin','Harvest.Year'])

In [ ]:
result['Country.of.Origin'].value_counts().plot(kind='bar', title= 'Distribution of coffee' ,  rot=90);

# Preprocessing

In [ ]:
result['Total.Cup.Points'].astype(int)


In [ ]:
def grading_coffee(x):

    if x>=80:
        return 'speciality'
    elif x>=70:
        return 'commercial+'

    

In [ ]:
#result=result.rename(columns={'Total.Cup.Points':'Total_Cup_Points'}, inplace=True)
#result.columns[18] = "Total_Cup_Points"
result1 = result.rename(columns={'Total.Cup.Points': 'Total_Cup_Points'})

In [ ]:
result1

In [ ]:
result1['coffee_grade']=result1.Total_Cup_Points.apply(grading_coffee)

In [ ]:
result1

In [ ]:
result1.groupby(['coffee_grade','Total_Cup_Points'])



In [ ]:
result1['coffee_grade'].value_counts()

In [ ]:
result1['coffee_grade'].value_counts().plot(kind='bar', title= 'Distribution coffee grade' ,  rot=0);

# Model


In [ ]:
df_train,df_test=train_test_split(result1, test_size=.2, random_state=199)

In [ ]:
X_train = df_train.loc[:,['Aroma', 'Flavor', 'Aftertaste','Acidity'
                          ,'Body','Balance','Uniformity','Clean.Cup',
                          'Sweetness','Cupper.Points']]

y_train = df_train['coffee_grade']

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
df_val.shape

Cross Validation

In [ ]:
x = df_val.loc[:,['Aroma', 'Flavor', 'Aftertaste','Acidity'
                          ,'Body','Balance','Uniformity','Clean.Cup',
                          'Sweetness','Cupper.Points']]
y = df_val['coffee_grade']

In [ ]:
from collections import Counter
counter = Counter(y)
for k, v in counter.items():
    dist = v / len(y) * 100 
    print(f"Class= {k}, n={v} ({dist}%)")

In [ ]:
plt.figure(1, figsize=(16,8))
plt.bar(counter.keys(),counter.values())

# SMOTE

In [ ]:
#Fixing the imbalance

In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE(random_state = 0)
x, y = oversample.fit_resample(x, y)

In [ ]:
from collections import Counter
counter = Counter(y)
for k, v in counter.items():
    dist = v / len(y) * 100 
    print(f"Class= {k}, n={v} ({dist}%)")

In [ ]:
plt.figure(1, figsize=(16,8))
plt.bar(counter.keys(),counter.values())

# LogisticRegression

In [ ]:
lm = LogisticRegression(C=0.5)
scores2 = []
pre_score2 = []
recall_score2 = []
kfold = KFold(n_splits=5)

for train_ix, val_ix in kfold.split(x, y):
    train_x, val_x = x.iloc[train_ix], x.iloc[val_ix]
    train_y, val_y = y.iloc[train_ix], y.iloc[val_ix]
    oversample = SMOTE(random_state = 0)
    train_x, train_y = oversample.fit_resample(train_x, train_y)
    lm.fit(train_x, train_y)
    y_pred =lm.predict(val_x)
    scores2.append((metrics.f1_score(val_y, y_pred, pos_label='3',average='micro')))
    pre_score2.append((metrics.precision_score(val_y, y_pred, pos_label='3',average='micro')))
    recall_score2.append((metrics.recall_score(val_y, y_pred, pos_label='3',average='micro')))
    
print("LogisticRegression score: \t")
print(sum(scores2) / len(scores2))
print("----------------")
conf_mat2 = confusion_matrix(val_y, y_pred)
print("LogisticRegression confusion matrix: \n",conf_mat2)
print("----------------")
print("LogisticRegression precision score")
print(sum(pre_score2) / len(pre_score2))
print("----------------")
print("LogisticRegression recall_score")
print(sum(recall_score2) / len(recall_score2))
print("----------------")

In [ ]:
y_pred_test =lm.predict(x)
print("LogisticRegression f1 score: \t")
print(metrics.f1_score(y, y_pred_test, pos_label='3',average='micro'))
print("----------------")
print("LogisticRegression precision score")
print(metrics.precision_score(y, y_pred_test, pos_label='3',average='micro'))
print("----------------")
print("LogisticRegression recall_score")
print((metrics.recall_score(y, y_pred_test, pos_label='3',average='micro')))

# KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
scores3 = []
pre_score3 = []
recall_scor3 = []
kfold = KFold(n_splits=5)

for train_ix, val_ix in kfold.split(x, y):

    train_x, val_x = x.iloc[train_ix], x.iloc[val_ix]
    train_y, val_y = y.iloc[train_ix], y.iloc[val_ix]
    
    
    oversample = SMOTE(random_state = 0)
    train_x, train_y = oversample.fit_resample(train_x, train_y)
    knn.fit(train_x, train_y)
    y_pred =knn.predict(val_x)
    scores3.append(metrics.f1_score(val_y, y_pred, pos_label='3',average='micro'))
    pre_score3.append(metrics.precision_score(val_y, y_pred, pos_label='3',average='micro'))
    recall_scor3.append(metrics.recall_score(val_y, y_pred, pos_label='3',average='micro'))

print("kNN f1 score: \t")
print(sum(scores3) / len(scores3))
print("----------------")
conf_mat3 = confusion_matrix(val_y, y_pred)
print("kNN confusion matrix: \n",conf_mat3)
print("----------------")
print("KNN precision score")
print(sum(pre_score3) / len(pre_score3))
print("----------------")
print("KNN recall_score")
print(sum(recall_scor3) / len(recall_scor3))
print("----------------")

In [ ]:
model1 = KNeighborsClassifier(n_neighbors=5)
scores = cross_val_score(model1, x, y, cv=5, n_jobs=-1, scoring='f1_macro')
print(scores)

In [ ]:
print(scores.mean())

# Decision Tree

In [ ]:
de =  DecisionTreeClassifier(max_depth=4)
scores1 = []
pre_score1 = []
recall_score1 = []
kfold = KFold(n_splits=5)

for train_ix, val_ix in kfold.split(x, y):
    train_X, val_X = x.iloc[train_ix], x.iloc[val_ix]
    train_y, val_y = y.iloc[train_ix], y.iloc[val_ix]
    oversample = SMOTE(random_state = 0)
    train_X, train_y = oversample.fit_resample(train_X, train_y)
    de.fit(train_X, train_y)
    y_pred =de.predict(val_X)
    scores1.append((metrics.f1_score(val_y, y_pred, pos_label='3',average='micro')))
    pre_score1.append((metrics.precision_score(val_y, y_pred, pos_label='3',average='micro')))
    recall_score1.append((metrics.recall_score(val_y, y_pred, pos_label='3',average='micro'))) 
    
print("Decision Tree score: \t")
print(sum(scores1) / len(scores1))
print("----------------")
conf_mat1 = confusion_matrix(val_y, y_pred)
print("Decision Tree confusion matrix: \n",conf_mat1)
print("----------------")
print("Decision Tree precision score")
print(sum(pre_score1) / len(pre_score1))
print("----------------")
print("Decision Tree recall_score")
print(sum(recall_score1) / len(recall_score1))
print("----------------")

In [ ]:
y_pred_test =de.predict(x)
print("Decision Tree f1 score: \t")
print(metrics.f1_score(y, y_pred_test, pos_label='3',average='micro'))
print("----------------")
print("Decision Tree precision score")
print(metrics.precision_score(y, y_pred_test, pos_label='3',average='micro'))
print("----------------")
print("Decision Tree recall_score")
print((metrics.recall_score(y, y_pred_test, pos_label='3',average='micro')))

# Random Forest

In [ ]:
rm =  RandomForestClassifier(n_estimators=100)
scores = []
pre_score = []
recall_score = []
kfold = KFold(n_splits=5)

for train_ix, val_ix in kfold.split(x, y):
    train_X, val_X = x.iloc[train_ix], x.iloc[val_ix]
    train_y, val_y = y.iloc[train_ix], y.iloc[val_ix]
    oversample = SMOTE(random_state = 0)
    train_X, train_y = oversample.fit_resample(train_X, train_y)
    rm.fit(train_X, train_y)
    y_pred =rm.predict(val_X)
    scores.append((metrics.f1_score(val_y, y_pred, pos_label='3',average='micro')))
    pre_score.append((metrics.precision_score(val_y, y_pred, pos_label='3',average='micro')))
    recall_score.append((metrics.recall_score(val_y, y_pred, pos_label='3',average='micro'))) 
    
print("Decision Tree F1: \t")
print(sum(scores) / len(scores))
print("----------------")
conf_mat = confusion_matrix(val_y, y_pred)
print("Decision Tree confusion matrix: \n",conf_mat)
print("----------------")
print("Decision Tree precision score")
print(sum(pre_score) / len(pre_score))
print("----------------")
print("Decision Tree recall_score")
print(sum(recall_score) / len(recall_score))

In [ ]:
y_pred_test =rm.predict(x)
print("Random Forest f1 score: \t")
print(metrics.f1_score(y, y_pred_test, pos_label='3',average='micro'))
print("----------------")
print("Random Forest precision score")
print(metrics.precision_score(y, y_pred_test, pos_label='3',average='micro'))
print("----------------")
print("Random Forest recall_score")
print((metrics.recall_score(y, y_pred_test, pos_label='3',average='micro')))

# Voting

In [ ]:
seed = 765
score_vot = []
pre_score_vot = []
recall_score_vot = []
log_clf = KNeighborsClassifier()
rnd_clf = RandomForestClassifier(random_state=seed)
dt_clf = DecisionTreeClassifier(random_state=seed)
voting_clf = VotingClassifier(estimators=[('lr', log_clf),('df', dt_clf),('rf',rnd_clf)], voting='hard')
kfold = KFold(n_splits=5)

for train_ix, val_ix in kfold.split(x, y):
    train_X, val_X = x.iloc[train_ix], x.iloc[val_ix]
    train_y, val_y = y.iloc[train_ix], y.iloc[val_ix]
    oversample = SMOTE(random_state = 0)
    train_X, train_y = oversample.fit_resample(train_X, train_y)
    voting_clf.fit(train_X, train_y)
    y_pred_tv_vclf =voting_clf.predict(val_X)
    score_vot.append(metrics.f1_score(val_y, y_pred_tv_vclf, pos_label='3',average='micro'))
    pre_score_vot.append(metrics.precision_score(val_y, y_pred_tv_vclf, pos_label='3',average='micro'))
    recall_score_vot.append(metrics.recall_score(val_y, y_pred_tv_vclf, pos_label='3',average='micro'))
    
print("voting F1: \t")
print(sum(score_vot) / len(score_vot))
print("----------------")
print("voting precision score")
print(sum(pre_score_vot) / len(pre_score_vot))
print("----------------")
print("votingrecall_score")
print(sum(recall_score_vot) / len(recall_score_vot))

In [ ]:
y_pred_test =voting_clf.predict(x)
print("voting f1 score: \t")
print(metrics.f1_score(y, y_pred_test, pos_label='3',average='micro'))
print("----------------")
print("voting precision score")
print(metrics.precision_score(y, y_pred_test, pos_label='3',average='micro'))
print("----------------")
print("voting recall_score")
print((metrics.recall_score(y, y_pred_test, pos_label='3',average='micro')))

# Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
bg = BaggingClassifier(RandomForestClassifier(), n_estimators=10, max_samples=0.5, bootstrap=True,n_jobs=-1)
scores_b = []
pre_score_b = []
recall_score_b = []
kfold = KFold(n_splits=5)
for train_ix, val_ix in kfold.split(x, y):
    train_X, val_X = x.iloc[train_ix], x.iloc[val_ix]
    train_y, val_y = y.iloc[train_ix], y.iloc[val_ix]
    oversample = SMOTE(random_state = 0)
    train_X, train_y = oversample.fit_resample(train_X, train_y)
    bg.fit(train_X, train_y)
    y_pred_tv_bg1 =bg.predict(val_X)
    scores_b.append(metrics.recall_score(val_y, y_pred_tv_bg1, pos_label='3',average='micro'))
    pre_score_b.append(metrics.precision_score(val_y, y_pred_tv_bg1, pos_label='3',average='micro'))
    recall_score_b.append(metrics.recall_score(val_y, y_pred_tv_bg1, pos_label='3',average='micro'))
print("Bagging F1: \t")
print(sum(scores_b) / len(scores_b))
print("Bagging precision score")
print(sum(pre_score_b) / len(pre_score_b))
print("----------------")
print("Bagging recall_score")
print(sum(recall_score_b) / len(recall_score_b))

In [ ]:
y_pred_test =bg.predict(x)
print("Bagging f1 score: \t")
print(metrics.f1_score(y, y_pred_test, pos_label='3',average='micro'))
print("----------------")
print("Bagging precision score")
print(metrics.precision_score(y, y_pred_test, pos_label='3',average='micro'))
print("----------------")
print("Bagging recall_score")
print((metrics.recall_score(y, y_pred_test, pos_label='3',average='micro')))